# Prueba de Datos Peliculas

OJO: Este script está pensado para ejecutarse de modo local:
Paso 1: ejecutar jupyter notebooks con permisospara collaboratory mediante cmd(
jupyter notebook 
  --NotebookApp.allow_origin='https://colab.research.google.com' 
  --port=8888 
  --NotebookApp.port_retries=0
  )
Apretar reconectar, luego conectar a un entorno de ejecución local

In [ ]:
import pymysql
import warnings
import pandas as pd

In [ ]:
connection = pymysql.connect(host="mel.analytics.cl", user="jterrazas",password="11jterrazas54",db="Cineplanet_Chile", port=33060)

query = "SELECT * FROM PELICULAS_VIGENTES;"
peliculas = pd.read_sql(query, connection)

In [ ]:
peliculas.shape

(12, 2)

In [ ]:
peliculas.head()

,id,titulo
0,HO00000490,AVENGERS: ENDGAME EXTENDIDA
1,HO00000494,CINE ARTE 2019: EL PADRINO
2,HO00000495,METALLICA & SAN FRANCISCO SYMPHONY: S&M2
3,HO00000472,LA NOCHE DE LAS NERDS
4,HO00000457,PAW PATROL: MIGHTY PUPS


In [ ]:
import re
import time
import requests
from bs4 import BeautifulSoup
#from omdb import OMDBClient
import urllib.request, urllib.parse, urllib.error
from urllib.parse import urlencode, urlparse
import json

def search_movie(id_movie):
	serviceurl = 'http://www.omdbapi.com/?'
	apikey = '&apikey='+"e0dd629a" # el segundo numero es la api key
	if len(id_movie) < 1 or id_movie=='quit': 
		print("Goodbye now...")
		return None
	try:
		url = serviceurl + urllib.parse.urlencode({'i': id_movie})+apikey
		#print(f'Retrieving the data of "{id_movie}" now... ')
		uh = urllib.request.urlopen(url)
		data = uh.read()
		json_data=json.loads(data)
		return(json_data)
	except urllib.error.URLError as e:
		#print(f"ERROR: {e.reason}")
		return None

def getOmdData(title):
	actual_title = re.sub("[*].*$","",title)
	actual_title_list =  actual_title.split(" ")
	actual_title_query = "%20".join(actual_title_list)
	actual_title_query = "https://www.imdb.com/find?s=all&q="+actual_title_query+"&ref_=nv_sr_sm"
	
	time.sleep(1)
	req = requests.get(actual_title_query)
	soup = BeautifulSoup(req.text, "lxml")
	actual_title_webscrape = soup.select('.primary_photo a')
	movie_id = str(actual_title_webscrape).split('/')[2]
	actual_title_params = search_movie(movie_id)

	return actual_title_params


def create_dataset(titles):
	dataset = pd.DataFrame(columns=['Actors','Country','Director'
		,'Genre','Language','Metascore','Plot','Production','Released'
		,'Runtime','Title','Year','imdbRating','imdbVotes'])#,'PEL_ID'])
	not_founds=[]
	titulo_original=[]
	i=1
	tiempo_ejecucion = []
	for tit in titles:
		time1 = time.time()
		print(i,". ", tit)
		try:
			pelicula = getOmdData(tit)
			bla = pd.DataFrame({'Actors': [pelicula['Actors']]
				,'Country':[pelicula['Country']]
				,'Director' : [pelicula['Director']]
				,'Genre':[pelicula['Genre']]
				,'Language':[pelicula['Language']]
				,'Metascore':[pelicula['Metascore']]
				,'Plot':[pelicula['Plot']]
				,'Production':[pelicula['Production']]
				,'Released':[pelicula['Released']]
				,'Runtime':[pelicula['Runtime']]
				,'Title':[pelicula['Title']]
				,'Year':[pelicula['Year']]
				,'imdbRating':[pelicula['imdbRating']]
				,'imdbVotes':[pelicula['imdbVotes']]
				#,'PEL_ID': [peliculas.PEL_ID[titles==tit].iloc[0]]
				})
			titulo_original.append(pelicula['Title'])
			dataset = dataset.append(bla)
		except:
			titulo_original.append('NO ENCONTRADO')
			not_founds.append(tit)
		time2 = time.time()
		tiempo_ejecucion.append(time2 - time1)
		i=i+1

	dataset.Title = [re.sub(r'[^a-zA-Z\s]', "", c).lower() for c in dataset.Title]
	dataset = dataset.reset_index(drop=True)
	print("Fin de busqueda de informacion sobre peliculas")
	print("No Encontrados: ", not_founds)
	print()
	return dataset

In [ ]:
dataset = create_dataset(peliculas.titulo)

1 .  AVENGERS: ENDGAME EXTENDIDA
2 .  CINE ARTE 2019: EL PADRINO
3 .  METALLICA & SAN FRANCISCO SYMPHONY: S&M2
4 .  LA NOCHE DE LAS NERDS
5 .  PAW PATROL: MIGHTY PUPS
6 .  RÁPIDOS Y FURIOSOS: HOBBS & SHAW
7 .  LA VIDA SECRETA DE TUS MASCOTAS 2
8 .  EL REY LEÓN
9 .  ANNABELLE 3: VIENE A CASA
10 .  TOY STORY 4
11 .  SPIDER-MAN LEJOS DE CASA
12 .  EL MUÑECO DIABÓLICO
Fin de busqueda de informacion sobre peliculas
No Encontrados:  ['AVENGERS: ENDGAME EXTENDIDA', 'CINE ARTE 2019: EL PADRINO', 'METALLICA & SAN FRANCISCO SYMPHONY: S&M2', 'PAW PATROL: MIGHTY PUPS']



In [ ]:
dataset.head()

,Actors,Country,Director,Genre,Language,Metascore,Plot,Production,Released,Runtime,Title,Year,imdbRating,imdbVotes
0,"Kaitlyn Dever, Beanie Feldstein, Jessica Willi...",USA,Olivia Wilde,Comedy,"English, Mandarin, Spanish",84,"On the eve of their high school graduation, tw...",United Artists,24 May 2019,102 min,booksmart,2019,7.4,"28,104"
1,"Dwayne Johnson, Jason Statham, Idris Elba, Van...","UK, USA",David Leitch,"Action, Adventure",English,N/A,Lawman Luke Hobbs and outcast Deckard Shaw for...,Universal Pictures,01 Aug 2019,135 min,fast furious presents hobbs shaw,2019,N/A,N/A
2,"Patton Oswalt, Kevin Hart, Harrison Ford, Eric...","France, Japan, USA","Chris Renaud, Jonathan del Val(co-director)","Animation, Adventure, Comedy, Family",English,55,Continuing the story of Max and his pet friend...,Universal Pictures,07 Jun 2019,86 min,the secret life of pets,2019,6.6,"10,610"
3,"Chiwetel Ejiofor, John Oliver, James Earl Jone...",USA,Jon Favreau,"Animation, Adventure, Drama, Family, Musical","English, Hindi, Tamil, Telugu",55,"After the murder of his father, a young lion p...",Walt Disney Pictures,19 Jul 2019,118 min,the lion king,2019,7.2,"50,414"
4,"Vera Farmiga, Patrick Wilson, Mckenna Grace, M...",USA,Gary Dauberman,"Horror, Mystery, Thriller",English,53,While babysitting the daughter of Ed and Lorra...,New Line Cinema,26 Jun 2019,106 min,annabelle comes home,2019,6.3,"17,605"
